In [1]:
import tensorflow as tf

**Input N Векторов Размерности K;
M внутренний размер attention**

w_key, w_query, w_value: KxM

key, query, value: NxM

q * k.T: NxN

softmax(q * k.T): NxN

weighted_values: NxNxM

output: NxM

Usually M = K, so self-attention blocks can be stacked

In [2]:
# N = 2, K = 4, M = 3

x = tf.constant([
  [1, 0, 1, 0],
  [1, 1, 1, 1],
 ], dtype=tf.float32)

w_key = tf.constant([
  [0, 0, 1],
  [1, 1, 0],
  [0, 1, 0],
  [1, 1, 0]
], dtype=tf.float32)

w_query = tf.constant([
  [0, 0, 1],
  [1, 1, 0],
  [0, 0, 1],
  [0, 1, 1]
], dtype=tf.float32)

w_value = tf.constant([
  [0, 2, 0],
  [0, 3, 0],
  [1, 0, 3],
  [1, 1, 0]
], dtype=tf.float32)

In [3]:
keys = x @ w_key
querys = x @ w_query
values = x @ w_value

print('keys', keys)
print('querys', querys)
print('values', values)

keys tf.Tensor(
[[0. 1. 1.]
 [2. 3. 1.]], shape=(2, 3), dtype=float32)
querys tf.Tensor(
[[0. 0. 2.]
 [1. 2. 3.]], shape=(2, 3), dtype=float32)
values tf.Tensor(
[[1. 2. 3.]
 [2. 6. 3.]], shape=(2, 3), dtype=float32)


In [4]:
attn_scores = querys @ tf.transpose(keys) #tf.matmul(querys, keys, transpose_b=True)
attn_scores

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 2.,  2.],
       [ 5., 11.]], dtype=float32)>

In [5]:
attn_scores_softmax = tf.nn.softmax(attn_scores, axis=-1)
attn_scores_softmax

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[0.5       , 0.5       ],
       [0.00247262, 0.9975274 ]], dtype=float32)>

In [6]:
# to ease calculation
attn_scores_softmax = tf.round(attn_scores_softmax * 10) / 10

In [7]:
#tf.matmul(attn_scores_softmax, values)
weighted_values = values[:,None] * tf.transpose(attn_scores_softmax)[:,:,None]
outputs = tf.math.reduce_sum(weighted_values, axis=0)
outputs

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1.5, 4. , 3. ],
       [2. , 6. , 3. ]], dtype=float32)>